In [1]:
"""
BUSINESS INTELLIGENCE PROJECT - ETL PIPELINE
Team: Mariem & Rihem
Dataset: Retail Sales
"""

# 1. Imports

import pandas as pd
import numpy as np
from datetime import datetime
import os
import sqlite3  # Added for database loading

print("📊 Starting ETL Process...")

# 2. Data loading from train.csv

df = pd.read_csv('train.csv')
print(f"✅ Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# 3. Data exploration and analysis

print("\n📈 Data Exploration:")
print("-" * 30)

# Basic info
print(f"Date Range: {df['Order Date'].min()} to {df['Order Date'].max()}")
print(f"Total Sales: ${df['Sales'].sum():,.2f}")
print(f"Number of Customers: {df['Customer ID'].nunique()}")
print(f"Number of Products: {df['Product ID'].nunique()}")

# Check what columns we actually have
print("\n📋 Available Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2}. {col}")

# Check data types
print(f"\nData Types:")
print(df.dtypes)

# Check for missing values
missing = df.isnull().sum()
if missing.sum() > 0:
    print("\n⚠️ Missing Values Found:")
    for col, count in missing.items():
        if count > 0:
            print(f"   {col}: {count} missing values")
else:
    print("\n✅ No missing values found!")

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\n📝 Duplicate Rows: {duplicates}")

# 4. Data cleaning (dates, duplicates, missing values)

print("\n🧹 Data Cleaning:")
print("-" * 30)

# Fix date columns
print("Converting dates...")
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y')

# Remove duplicates if any
if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Removed {duplicates} duplicate rows")

# Fix text columns (remove extra spaces)
text_columns = ['Customer Name', 'Product Name', 'City', 'State']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].str.strip()

# Handle missing values (simple approach)
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':  # Text columns
            df[col].fillna('Unknown', inplace=True)
        else:  # Numeric columns
            df[col].fillna(df[col].median(), inplace=True)

print("✅ Data cleaning complete!")

# 5. Feature engineering (new business columns)

print("\n🎯 Creating New Columns:")
print("-" * 30)

# Time-based columns
df['Order_Year'] = df['Order Date'].dt.year
df['Order_Month'] = df['Order Date'].dt.month
df['Order_Quarter'] = df['Order Date'].dt.quarter
df['Order_Day'] = df['Order Date'].dt.day
df['Day_of_Week'] = df['Order Date'].dt.day_name()

# Business metrics
df['Shipping_Days'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Shipping_On_Time'] = df['Shipping_Days'] <= 7  # Assuming 7 days is standard

# Month-Year combination for time series
df['Year_Month'] = df['Order Date'].dt.strftime('%Y-%m')

print(f"Added new columns: {[col for col in df.columns if col not in ['Order Date', 'Ship Date', 'Sales']]}")


# 6.  Saving cleaned data to CSV files

print("\n💾 Saving Data:")
print("-" * 30)

# Create output folder
os.makedirs('output', exist_ok=True)

# Save cleaned data
df.to_csv('output/cleaned_sales_data.csv', index=False)
print("✅ Cleaned data saved as: 'output/cleaned_sales_data.csv'")

# Save a smaller sample for testing
df.head(1000).to_csv('output/sample_data.csv', index=False)
print("✅ Sample data saved as: 'output/sample_data.csv'")

# 7.  DATABASE LOADING (to SQLite database)

print("\n🗄️  Loading Data to Database:")
print("-" * 30)

# Create database directory
os.makedirs('database', exist_ok=True)

# Connect to SQLite database (creates it if doesn't exist)
db_path = 'database/sales_database.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Connected to database: {db_path}")

# Load data from DataFrame to database
print("Loading data into database...")
df.to_sql('sales_transactions', conn, if_exists='replace', index=False)

# Verify the load
cursor.execute("SELECT COUNT(*) FROM sales_transactions")
row_count = cursor.fetchone()[0]
print(f"✅ {row_count:,} rows loaded into database")

# Check what columns actually exist in the table
cursor.execute("PRAGMA table_info(sales_transactions)")
columns_info = cursor.fetchall()
column_names = [info[1] for info in columns_info]

print(f"\n📋 Columns in database table:")
for i, col in enumerate(column_names, 1):
    print(f"   {i:2}. {col}")

# Check if specific columns exist
has_profit = 'Profit' in column_names or 'profit' in column_names
has_postal_code = 'Postal Code' in column_names or 'Postal_Code' in column_names or 'postal_code' in column_names
has_segment = 'Segment' in column_names or 'segment' in column_names

print(f"\n🔍 Column check:")
print(f"   Has Profit column: {has_profit}")
print(f"   Has Postal Code column: {has_postal_code}")
print(f"   Has Segment column: {has_segment}")

# Create summary table based on available columns
print("\nCreating sales summary table...")

# Build the summary query dynamically based on available columns
summary_query = '''
CREATE TABLE IF NOT EXISTS sales_summary AS
SELECT 
    Order_Year,
    Order_Month,
'''

# Add Category if it exists
if 'Category' in column_names or 'category' in column_names:
    summary_query += '    Category,\n'
else:
    summary_query += "    'All' as Category,\n"

# Add Region if it exists  
if 'Region' in column_names or 'region' in column_names:
    summary_query += '    Region,\n'
else:
    summary_query += "    'All' as Region,\n"

# Add common metrics
summary_query += '''    COUNT(*) as order_count,
    SUM(Sales) as total_sales,
    AVG(Sales) as avg_order_value
'''

# Add Profit if it exists
if has_profit:
    summary_query += '    , SUM(Profit) as total_profit\n'

summary_query += '''FROM sales_transactions
GROUP BY Order_Year, Order_Month
'''

# Add Category and Region to GROUP BY if they exist
if 'Category' in column_names or 'category' in column_names:
    summary_query = summary_query.replace('GROUP BY Order_Year, Order_Month', 
                                         'GROUP BY Order_Year, Order_Month, Category')
if 'Region' in column_names or 'region' in column_names:
    summary_query = summary_query.replace('GROUP BY Order_Year, Order_Month', 
                                         'GROUP BY Order_Year, Order_Month, Region')

# Execute the dynamic query
try:
    cursor.execute('DROP TABLE IF EXISTS sales_summary')
    cursor.execute(summary_query)
    print("✅ Sales summary table created successfully!")
    
    # Show summary table info
    cursor.execute("SELECT COUNT(*) FROM sales_summary")
    summary_count = cursor.fetchone()[0]
    print(f"   Summary rows: {summary_count}")
    
    cursor.execute("PRAGMA table_info(sales_summary)")
    summary_columns = cursor.fetchall()
    print(f"   Summary columns: {[col[1] for col in summary_columns]}")
    
except Exception as e:
    print(f"⚠️  Could not create summary table: {e}")
    print("Creating basic summary table instead...")
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sales_summary AS
        SELECT 
            Order_Year,
            Order_Month,
            COUNT(*) as order_count,
            SUM(Sales) as total_sales,
            AVG(Sales) as avg_order_value
        FROM sales_transactions
        GROUP BY Order_Year, Order_Month
    ''')
    print("✅ Basic sales summary table created")

# Save and close
conn.commit()
conn.close()

print("✅ Database connection closed successfully!")
print(f"📁 Database file saved: {db_path}")
print(f"📊 Database size: {os.path.getsize(db_path) / 1024:.1f} KB")


# 8.  Data dictionary creation

print("\n📖 Creating Data Dictionary...")

data_dict = []

for column in df.columns:
    # Get basic info
    col_info = {
        'Column Name': column,
        'Data Type': str(df[column].dtype),
        'Unique Values': df[column].nunique(),
        'Missing Values': df[column].isnull().sum()
    }
    
    # Add description based on column name
    if 'Date' in column:
        col_info['Description'] = 'Date information'
    elif 'Sales' in column:
        col_info['Description'] = 'Sales amount in USD'
    elif 'Customer' in column:
        col_info['Description'] = 'Customer information'
    elif 'Product' in column:
        col_info['Description'] = 'Product information'
    elif 'Ship' in column:
        col_info['Description'] = 'Shipping information'
    elif 'Region' in column or 'State' in column or 'City' in column:
        col_info['Description'] = 'Geographic information'
    else:
        col_info['Description'] = 'Other information'
    
    # Add sample value
    if len(df[column]) > 0:
        sample = df[column].iloc[0]
        if pd.isna(sample):
            col_info['Sample Value'] = 'N/A'
        else:
            col_info['Sample Value'] = str(sample)[:50]  # First 50 chars
    else:
        col_info['Sample Value'] = 'N/A'
    
    data_dict.append(col_info)

# Save data dictionary
dict_df = pd.DataFrame(data_dict)
dict_df.to_csv('output/data_dictionary.csv', index=False)
dict_df.to_excel('output/data_dictionary.xlsx', index=False)
print("✅ Data dictionary saved as CSV and Excel!")

# 9. Final summary

print("\n" + "="*50)
print("🎉 ETL PROCESS COMPLETE!")
print("="*50)

print(f"\n📊 Final Dataset Summary:")
print(f"   • Total Rows: {df.shape[0]}")
print(f"   • Total Columns: {df.shape[1]}")
print(f"   • Date Range: {df['Order Date'].min().date()} to {df['Order Date'].max().date()}")
print(f"   • Total Sales: ${df['Sales'].sum():,.2f}")
print(f"   • Average Order Value: ${df['Sales'].mean():,.2f}")

# Check and display other metrics if they exist
if 'Region' in df.columns:
    print(f"   • Number of Regions: {df['Region'].nunique()}")
if 'Category' in df.columns:
    print(f"   • Number of Categories: {df['Category'].nunique()}")
if 'Profit' in df.columns:
    print(f"   • Total Profit: ${df['Profit'].sum():,.2f}")

print(f"\n📁 Files Created:")
print(f"   1. output/cleaned_sales_data.csv")
print(f"   2. output/sample_data.csv")
print(f"   3. output/data_dictionary.csv")
print(f"   4. output/data_dictionary.xlsx")
print(f"   5. database/sales_database.db (SQLite Database)")

print(f"\n🗄️  Database Information:")
print(f"   • Main Table: sales_transactions ({row_count:,} rows)")
print(f"   • Summary Table: sales_summary")
print(f"   • Total columns in dataset: {len(df.columns)}")

print("\n✅ Ready for Data Modeling in Power BI!")

📊 Starting ETL Process...
✅ Data loaded: 9800 rows, 18 columns

📈 Data Exploration:
------------------------------
Date Range: 01/01/2018 to 31/12/2017
Total Sales: $2,261,536.78
Number of Customers: 793
Number of Products: 1861

📋 Available Columns:
    1. Row ID
    2. Order ID
    3. Order Date
    4. Ship Date
    5. Ship Mode
    6. Customer ID
    7. Customer Name
    8. Segment
    9. Country
   10. City
   11. State
   12. Postal Code
   13. Region
   14. Product ID
   15. Category
   16. Sub-Category
   17. Product Name
   18. Sales

Data Types:
Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code      float64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales         

C:\Users\Mega Pc\AppData\Local\Temp\ipykernel_21208\1183048233.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


Added new columns: ['Row ID', 'Order ID', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Order_Year', 'Order_Month', 'Order_Quarter', 'Order_Day', 'Day_of_Week', 'Shipping_Days', 'Shipping_On_Time', 'Year_Month']

💾 Saving Data:
------------------------------
✅ Cleaned data saved as: 'output/cleaned_sales_data.csv'
✅ Sample data saved as: 'output/sample_data.csv'

🗄️  Loading Data to Database:
------------------------------
Connected to database: database/sales_database.db
Loading data into database...
✅ 9,800 rows loaded into database

📋 Columns in database table:
    1. Row ID
    2. Order ID
    3. Order Date
    4. Ship Date
    5. Ship Mode
    6. Customer ID
    7. Customer Name
    8. Segment
    9. Country
   10. City
   11. State
   12. Postal Code
   13. Region
   14. Product ID
   15. Category
   16. Sub-Category
   17. Product Name
   18. Sales
   19. Ord

In [3]:
#  DATA MODELING 

print("📊 STARTING DATA MODELING PHASE...")
print("="*50)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# 1. Load cleaned data

print("\n1. 📥 Loading cleaned data...")
try:
    df = pd.read_csv('output/cleaned_sales_data.csv')
    print(f"   ✅ Data loaded successfully: {len(df)} rows")
    print(f"   📋 Available columns: {list(df.columns)}")
except FileNotFoundError:
    print("   ❌ Error: cleaned_sales_data.csv not found. Run ETL first!")
    exit()

# Check what columns we actually have
print("\n   🔍 Checking column availability...")
available_cols = df.columns.tolist()
print(f"   Available columns: {available_cols}")

# CONVERT DATE COLUMNS FROM STRING TO DATETIME
print("\n   Converting date columns to datetime...")
# First, identify which date columns exist
date_cols = [col for col in available_cols if 'date' in col.lower() or 'Date' in col]
for col in date_cols:
    try:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"   Converted {col} to datetime")
    except:
        print(f"   Could not convert {col} to datetime")

print(f"\n   ✅ Loaded: {len(df)} rows, {len(df.columns)} columns")
if 'Order Date' in df.columns:
    print(f"   Date range: {df['Order Date'].min().date()} to {df['Order Date'].max().date()}")

# 2. Create dimension table

print("\n2. 🔧 Creating Dimension Tables...")

# Dim_Date - Date Dimension
print("   Creating Dim_Date...")

# Determine date range from available data
if 'Order Date' in df.columns and df['Order Date'].notna().any():
    min_date = df['Order Date'].min().date()
    max_date = df['Order Date'].max().date()
else:
    # Use default dates if no date column
    min_date = datetime(2020, 1, 1).date()
    max_date = datetime(2025, 12, 31).date()
    print(f"   ⚠️  Using default date range: {min_date} to {max_date}")

# Create date range
date_range = pd.date_range(
    start=min_date,
    end=max_date + timedelta(days=365),  # Add 1 year buffer
    freq='D'
)

dim_date = pd.DataFrame({
    'Date_Key': [int(d.strftime('%Y%m%d')) for d in date_range],
    'Date': date_range,
    'Year': [d.year for d in date_range],
    'Quarter': [f'Q{(d.month-1)//3 + 1}' for d in date_range],
    'Quarter_Num': [(d.month-1)//3 + 1 for d in date_range],
    'Month': [d.month for d in date_range],
    'Month_Name': [d.strftime('%B') for d in date_range],
    'Week_Num': [d.isocalendar()[1] for d in date_range],
    'Day_of_Week': [d.strftime('%A') for d in date_range],
    'Day_of_Week_Num': [d.weekday() + 1 for d in date_range],
    'Is_Weekend': [(d.weekday() >= 5) for d in date_range],
    'Is_Holiday': [False] * len(date_range),
    'Fiscal_Year': [d.year for d in date_range]
})

print(f"   ✅ Dim_Date created: {len(dim_date)} dates from {date_range[0].date()} to {date_range[-1].date()}")

# Dim_Product - Product Dimension (use available columns)
print("   Creating Dim_Product...")
# Find product-related columns
product_cols = []
for col in ['Product ID', 'Product Name', 'Category', 'Sub-Category', 'product_id', 'product_name']:
    if col in df.columns:
        product_cols.append(col)

if product_cols:
    dim_product = df[product_cols].copy()
    # Use the first column as the primary key
    pk_col = product_cols[0]
    dim_product = dim_product.drop_duplicates(subset=[pk_col])
    dim_product = dim_product.reset_index(drop=True)
    dim_product['Product_Key'] = range(1, len(dim_product) + 1)
    # Keep all product columns
    dim_product = dim_product[['Product_Key'] + product_cols]
    print(f"   ✅ Dim_Product created: {len(dim_product)} unique products")
else:
    print("   ⚠️  No product columns found, creating empty Dim_Product")
    dim_product = pd.DataFrame({'Product_Key': [], 'Product_ID': []})

# Dim_Customer - Customer Dimension (use available columns)
print("   Creating Dim_Customer...")
# Find customer-related columns
customer_cols = []
for col in ['Customer ID', 'Customer Name', 'Segment', 'customer_id', 'customer_name']:
    if col in df.columns:
        customer_cols.append(col)

if customer_cols:
    dim_customer = df[customer_cols].copy()
    # Use the first column as the primary key
    pk_col = customer_cols[0]
    dim_customer = dim_customer.drop_duplicates(subset=[pk_col])
    dim_customer = dim_customer.reset_index(drop=True)
    dim_customer['Customer_Key'] = range(1, len(dim_customer) + 1)
    # Keep all customer columns
    dim_customer = dim_customer[['Customer_Key'] + customer_cols]
    print(f"   ✅ Dim_Customer created: {len(dim_customer)} unique customers")
else:
    print("   ⚠️  No customer columns found, creating empty Dim_Customer")
    dim_customer = pd.DataFrame({'Customer_Key': [], 'Customer_ID': []})

# Dim_Region - Region Dimension (use available columns)
print("   Creating Dim_Region...")
# Find region-related columns
region_cols = []
for col in ['City', 'State', 'Region', 'Country', 'city', 'state', 'region']:
    if col in df.columns:
        region_cols.append(col)

if region_cols:
    dim_region = df[region_cols].copy()
    dim_region = dim_region.drop_duplicates()
    dim_region = dim_region.reset_index(drop=True)
    dim_region['Region_Key'] = range(1, len(dim_region) + 1)
    # Keep all region columns
    dim_region = dim_region[['Region_Key'] + region_cols]
    print(f"   ✅ Dim_Region created: {len(dim_region)} unique locations")
else:
    print("   ⚠️  No region columns found, creating empty Dim_Region")
    dim_region = pd.DataFrame({'Region_Key': [], 'Region': []})

# 3. Create fact table with foreign tables

print("\n3. 🎯 Creating Fact_Sales table...")

# Prepare fact table
fact_sales = df.copy()

# Create Date_Key for joining if we have Order Date
if 'Order Date' in df.columns:
    fact_sales['Date_Key'] = fact_sales['Order Date'].dt.strftime('%Y%m%d').astype(int)
elif 'order_date' in df.columns:
    fact_sales['Date_Key'] = pd.to_datetime(fact_sales['order_date']).dt.strftime('%Y%m%d').astype(int)
else:
    # Create a dummy Date_Key
    fact_sales['Date_Key'] = 20250101  # Default date

print("   Merging with dimension tables...")

# Merge with Product if possible
if 'Product_Key' in dim_product.columns and product_cols:
    product_lookup = dim_product.set_index(product_cols[0])['Product_Key'].to_dict()
    fact_sales['Product_Key'] = fact_sales[product_cols[0]].map(product_lookup)
else:
    fact_sales['Product_Key'] = -1  # Default unknown product

# Merge with Customer if possible
if 'Customer_Key' in dim_customer.columns and customer_cols:
    customer_lookup = dim_customer.set_index(customer_cols[0])['Customer_Key'].to_dict()
    fact_sales['Customer_Key'] = fact_sales[customer_cols[0]].map(customer_lookup)
else:
    fact_sales['Customer_Key'] = -1  # Default unknown customer

# Merge with Region if possible
if 'Region_Key' in dim_region.columns and region_cols:
    # Create composite key for region lookup
    dim_region['Location_Key'] = dim_region[region_cols[0]]
    if len(region_cols) > 1:
        for col in region_cols[1:]:
            dim_region['Location_Key'] = dim_region['Location_Key'] + '_' + dim_region[col]
    
    region_lookup = dim_region.set_index('Location_Key')['Region_Key'].to_dict()
    
    # Create same composite key in fact table
    fact_sales['Location_Key'] = fact_sales[region_cols[0]]
    if len(region_cols) > 1:
        for col in region_cols[1:]:
            fact_sales['Location_Key'] = fact_sales['Location_Key'] + '_' + fact_sales[col]
    
    fact_sales['Region_Key'] = fact_sales['Location_Key'].map(region_lookup)
else:
    fact_sales['Region_Key'] = -1  # Default unknown region

# Create Order_ID if not exists
if 'Order ID' not in fact_sales.columns and 'order_id' not in fact_sales.columns:
    fact_sales['Order_ID'] = range(1, len(fact_sales) + 1)
elif 'Order ID' in fact_sales.columns:
    fact_sales = fact_sales.rename(columns={'Order ID': 'Order_ID'})
elif 'order_id' in fact_sales.columns:
    fact_sales = fact_sales.rename(columns={'order_id': 'Order_ID'})

# Select available measure columns
available_measures = []
measure_mapping = {
    'Sales': 'Sales_Amount',
    'sales': 'Sales_Amount',
    'Quantity': 'Quantity',
    'quantity': 'Quantity',
    'Discount': 'Discount',
    'discount': 'Discount',
    'Profit': 'Profit_Amount',
    'profit': 'Profit_Amount',
    'Shipping_Days': 'Shipping_Days',
    'shipping_days': 'Shipping_Days'
}

for orig_col, new_name in measure_mapping.items():
    if orig_col in fact_sales.columns:
        fact_sales[new_name] = fact_sales[orig_col]
        available_measures.append(new_name)

# Ensure we have at least Sales_Amount
if 'Sales_Amount' not in fact_sales.columns and 'Sales' in fact_sales.columns:
    fact_sales['Sales_Amount'] = fact_sales['Sales']
    available_measures.append('Sales_Amount')

# Select columns for final fact table
fact_columns = ['Order_ID', 'Date_Key', 'Product_Key', 'Customer_Key', 'Region_Key']
fact_columns += available_measures

# Keep only columns that exist
fact_columns = [col for col in fact_columns if col in fact_sales.columns]
fact_sales = fact_sales[fact_columns]

print(f"   ✅ Fact_Sales created with {len(fact_sales)} rows and {len(fact_sales.columns)} columns")
print(f"   📊 Available measures: {available_measures}")

# Check for any missing foreign keys
missing_keys = fact_sales.isnull().sum()
if missing_keys.any():
    print(f"   ⚠️  Warning: Found missing values:")
    for col, count in missing_keys.items():
        if count > 0:
            print(f"      - {col}: {count} missing values ({count/len(fact_sales)*100:.1f}%)")
    
    # Fill missing values
    for col in ['Product_Key', 'Customer_Key', 'Region_Key']:
        if col in fact_sales.columns:
            fact_sales[col] = fact_sales[col].fillna(-1).astype(int)
    
    for col in available_measures:
        if col in fact_sales.columns:
            fact_sales[col] = fact_sales[col].fillna(0)
else:
    print("   ✅ All foreign keys are valid")

# 4. Save all tables

print("\n4. 💾 Saving all tables...")

# Create model directory
os.makedirs('model', exist_ok=True)

# Save dimension tables
dim_date.to_csv('model/Dim_Date.csv', index=False)
dim_product.to_csv('model/Dim_Product.csv', index=False)
dim_customer.to_csv('model/Dim_Customer.csv', index=False)
dim_region.to_csv('model/Dim_Region.csv', index=False)
fact_sales.to_csv('model/Fact_Sales.csv', index=False)

print("   ✅ Saved:")
print("      - model/Dim_Date.csv")
print("      - model/Dim_Product.csv")
print("      - model/Dim_Customer.csv")
print("      - model/Dim_Region.csv")
print("      - model/Fact_Sales.csv")

# 5. Create data model diagram

print("\n5. 📐 Creating Data Model Diagram...")

# Create ASCII-only diagram that won't cause encoding issues
diagram = f"""
====================================================================
                   STAR SCHEMA DATA MODEL
                   Retail Sales Analysis
====================================================================

RELATIONSHIPS:
- Fact_Sales.Date_Key -> Dim_Date.Date_Key
- Fact_Sales.Product_Key -> Dim_Product.Product_Key
- Fact_Sales.Customer_Key -> Dim_Customer.Customer_Key
- Fact_Sales.Region_Key -> Dim_Region.Region_Key

                  +-----------------+
                  |    Dim_Date     |
                  |   (Date_Key)    |
                  +--------+--------+
                           |
        +------------------+------------------+
        |                                     |
+-------+-------+                   +---------+-------+
|  Dim_Product  |                   |  Dim_Customer  |
| (Product_Key) |                   | (Customer_Key) |
+-------+-------+                   +---------+-------+
        |                                     |
        +------------------+------------------+
                           |
                  +--------+--------+
                  |    Dim_Region   |
                  |   (Region_Key)  |
                  +-----------------+

                           |
                  +--------+--------+
                  |   Fact_Sales    |
                  |     (Fact)      |
                  +-----------------+

TABLE STRUCTURE:
+----------------+---------------------+------------+------------+
| Table          | Primary Key         | Row Count  | Type       |
+----------------+---------------------+------------+------------+
| Fact_Sales     | Order_ID            | {len(fact_sales):,}     | Fact       |
| Dim_Date       | Date_Key            | {len(dim_date):,}     | Dimension  |
| Dim_Product    | Product_Key         | {len(dim_product):,}     | Dimension  |
| Dim_Customer   | Customer_Key        | {len(dim_customer):,}     | Dimension  |
| Dim_Region     | Region_Key          | {len(dim_region):,}     | Dimension  |
+----------------+---------------------+------------+------------+

FACT TABLE COLUMNS: {list(fact_sales.columns)}

DATA MODEL READY FOR POWER BI!
"""

# Save diagram with UTF-8 encoding to handle any special characters
with open('model/data_model_diagram.txt', 'w', encoding='utf-8') as f:
    f.write(diagram)

print("   ✅ Data model diagram saved: model/data_model_diagram.txt")


# 6. Define Dax measures based on available data

print("\n6. 📈 Defining DAX Measures...")

# Create measures based on what's available in fact_sales
measures_content = """============================================================
                   DAX MEASURES DEFINITIONS
============================================================
Project: Business Intelligence - Retail Sales Analysis
Team: Mariem & Rihem  # Fixed team name
Date: December 2025

REQUIRED MEASURES (At least 3 DAX measures):
"""

# Use a counter for sequential numbering
counter = 1

# Add measures based on available columns
if 'Sales_Amount' in fact_sales.columns:
    measures_content += f"""
{counter}. TOTAL SALES
   Formula:  Total_Sales = SUM(Fact_Sales[Sales_Amount])
   Type:     Aggregation (SUM)
   Use:      Primary KPI for sales performance
"""
    counter += 1

if 'Quantity' in fact_sales.columns:
    measures_content += f"""
{counter}. TOTAL QUANTITY
   Formula:  Total_Quantity = SUM(Fact_Sales[Quantity])
   Type:     Aggregation (SUM)
   Use:      Total items sold
"""
    counter += 1

if 'Profit_Amount' in fact_sales.columns:
    measures_content += f"""
{counter}. TOTAL PROFIT
   Formula:  Total_Profit = SUM(Fact_Sales[Profit_Amount])
   Type:     Aggregation (SUM)
   Use:      Total profit earned
"""
    counter += 1

if 'Sales_Amount' in fact_sales.columns:
    measures_content += f"""
{counter}. AVERAGE ORDER VALUE
   Formula:  Avg_Order_Value = AVERAGE(Fact_Sales[Sales_Amount])
   Type:     Aggregation (AVG)
   Use:      Customer spending analysis
"""
    counter += 1

if 'Sales_Amount' in fact_sales.columns:
    measures_content += f"""
{counter}. YEAR-OVER-YEAR GROWTH
   Formula:  Sales_YoY = 
             VAR CurrentYear = [Total_Sales]
             VAR PreviousYear = CALCULATE([Total_Sales], SAMEPERIODLASTYEAR(Dim_Date[Date]))
             RETURN DIVIDE(CurrentYear - PreviousYear, PreviousYear, 0)
   Type:     Time Intelligence (YOY)
   Use:      Annual performance comparison
"""
    counter += 1
    
    measures_content += f"""
{counter}. MONTH-OVER-MONTH GROWTH
   Formula:  Sales_MoM = 
             VAR CurrentMonth = [Total_Sales]
             VAR PreviousMonth = CALCULATE([Total_Sales], PREVIOUSMONTH(Dim_Date[Date]))
             RETURN DIVIDE(CurrentMonth - PreviousMonth, PreviousMonth, 0)
   Type:     Time Intelligence (MOM)
   Use:      Monthly trend analysis
"""
    counter += 1

if 'Discount' in fact_sales.columns:
    measures_content += f"""
{counter}. DISCOUNT RATE
   Formula:  Discount_Rate = AVERAGE(Fact_Sales[Discount])
   Type:     Aggregation (AVG)
   Use:      Average discount percentage
"""
    counter += 1

if 'Profit_Amount' in fact_sales.columns and 'Sales_Amount' in fact_sales.columns:
    measures_content += f"""
{counter}. PROFIT MARGIN
   Formula:  Profit_Margin = DIVIDE([Total_Profit], [Total_Sales])
   Type:     Ratio
   Use:      Profitability analysis
"""
    counter += 1

if 'Shipping_Days' in fact_sales.columns:
    measures_content += f"""
{counter}. AVERAGE SHIPPING DAYS
   Formula:  Avg_Shipping_Days = AVERAGE(Fact_Sales[Shipping_Days])
   Type:     Aggregation (AVG)
   Use:      Shipping performance
"""
    counter += 1

# Time intelligence measures (always added if sales exists)
if 'Sales_Amount' in fact_sales.columns:
    measures_content += f"""
ADDITIONAL TIME INTELLIGENCE MEASURES:

{counter}. YEAR-TO-DATE SALES
    Formula:  Sales_YTD = TOTALYTD([Total_Sales], Dim_Date[Date])
"""
    counter += 1
    
    measures_content += f"""
{counter}. MONTH-TO-DATE SALES
    Formula:  Sales_MTD = TOTALMTD([Total_Sales], Dim_Date[Date])
"""
    counter += 1
    
    measures_content += f"""
{counter}. QUARTER-TO-DATE SALES
    Formula:  Sales_QTD = TOTALQTD([Total_Sales], Dim_Date[Date])
"""
    counter += 1

measures_content += """
IMPLEMENTATION NOTES:
• All measures should be created in Power BI
• Date table (Dim_Date) must be marked as date table
• Relationships must be properly set up
• Use DIVIDE() to handle divide-by-zero errors
• Test measures with sample data

============================================================
"""

with open('model/DAX_Measures.txt', 'w', encoding='utf-8') as f:
    f.write(measures_content)

print("   ✅ DAX measures saved: model/DAX_Measures.txt")

# 7. Create one page summary

print("\n7. 📄 Creating One-Page Summary...")

one_page = f"""BUSINESS INTELLIGENCE PROJECT - DATA MODELING SUMMARY
===========================================================

PROJECT: Retail Sales Analysis
TEAM: Mariem & Rihem
DATE: {datetime.now().strftime('%Y-%m-%d')}

STAR SCHEMA IMPLEMENTED:
• Fact_Sales: Transactional data with {len(available_measures)} measures
• Dim_Date: Time intelligence with {len(dim_date.columns)} attributes
• Dim_Product: Product dimension with {len(dim_product)} products
• Dim_Customer: Customer dimension with {len(dim_customer)} customers
• Dim_Region: Region dimension with {len(dim_region)} locations

DATA MODEL STATISTICS:
• Total Records: {len(fact_sales) + len(dim_date) + len(dim_product) + len(dim_customer) + len(dim_region):,}
• Date Range: {min_date} to {max_date}
• Fact Table Columns: {list(fact_sales.columns)}

DAX MEASURES DEFINED:
Based on available data, the following measures are defined:
1. Total_Sales - SUM aggregation
2. Total_Quantity - SUM aggregation (if available)
3. Total_Profit - SUM aggregation (if available)
4. Sales_YoY - Year-over-year growth
5. Sales_MoM - Month-over-month growth
6. Profit_Margin - Financial ratio (if profit data available)
7. Additional time intelligence measures

READY FOR POWER BI DASHBOARD DEVELOPMENT:
✓ Data model created (star schema)
✓ Dimension tables ready
✓ Fact table with foreign keys
✓ DAX measures defined based on available data
✓ Documentation complete

NEXT STEPS: Load into Power BI and create:
1. Executive Summary Dashboard
2. Sales Trend Analysis
3. Product Performance Report
4. Customer Segmentation Analysis
===========================================================
"""

with open('model/OnePage_Summary.txt', 'w', encoding='utf-8') as f:
    f.write(one_page)

print("   ✅ One-page summary saved: model/OnePage_Summary.txt")


# 8. Create complete documentation

print("\n8. 📚 Creating Complete Documentation...")

readme_content = f"""# DATA MODEL DOCUMENTATION
## Business Intelligence Project - Week 2-3

### Project Information
- **Project**: Retail Sales Analysis
- **Team**: Rihem & Mariem
- **Course**: IT300 - Tunis Business School
- **Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

### Files in this directory:
1. Dim_Date.csv - Date dimension table
2. Dim_Product.csv - Product dimension table  
3. Dim_Customer.csv - Customer dimension table
4. Dim_Region.csv - Region dimension table
5. Fact_Sales.csv - Fact table with transactions
6. data_model_diagram.txt - Schema visualization
7. DAX_Measures.txt - Power BI measure definitions
8. OnePage_Summary.txt - Project summary

### Data Statistics:
- Total transactions: {len(fact_sales):,}
- Time period: {min_date} to {max_date}
- Available measures in fact table: {available_measures}
- Unique products: {len(dim_product):,}
- Unique customers: {len(dim_customer):,}
- Unique locations: {len(dim_region):,}

### How to use in Power BI:
1. Import all CSV files into Power BI
2. Create relationships between tables
3. Mark Dim_Date[Date] as Date Table
4. Create DAX measures from DAX_Measures.txt
5. Build dashboards and reports

### Project Status:
✅ ETL Completed
✅ Data Modeling Completed  
◻ Dashboard Development (Next)
◻ Insights & Reporting

For questions, contact the project team.
"""

with open('model/README.txt', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("   ✅ Documentation saved: model/README.txt")

# 9. Create csv files Read me

print("\n9. 📋 Creating CSV Files Documentation...")

csv_docs = f"""CSV FILES DOCUMENTATION
=======================

1. Fact_Sales.csv ({len(fact_sales):,} rows, {len(fact_sales.columns)} columns)
   Purpose: Main transactional data table
   Key columns:
   - Order_ID: Unique identifier for each order
   - Date_Key: Foreign key to date dimension
   - Product_Key: Foreign key to product dimension
   - Customer_Key: Foreign key to customer dimension
   - Region_Key: Foreign key to region dimension
   - Sales_Amount: Sales value in currency
   - Additional measures: {available_measures}

2. Dim_Date.csv ({len(dim_date):,} rows, {len(dim_date.columns)} columns)
   Purpose: Date dimension for time-based analysis
   Key columns:
   - Date_Key: Primary key (YYYYMMDD format)
   - Date: Full date
   - Year, Quarter, Month: Time hierarchies
   - Month_Name, Day_of_Week: Text representations
   - Is_Weekend: Boolean for weekend identification

3. Dim_Product.csv ({len(dim_product):,} rows, {len(dim_product.columns)} columns)
   Purpose: Product information and categorization
   Key columns:
   - Product_Key: Primary key
   - Product_ID: Original product identifier
   - Product_Name: Name of product
   - Category: Product category
   - Sub-Category: Product sub-category

4. Dim_Customer.csv ({len(dim_customer):,} rows, {len(dim_customer.columns)} columns)
   Purpose: Customer information and segmentation
   Key columns:
   - Customer_Key: Primary key
   - Customer_ID: Original customer identifier
   - Customer_Name: Name of customer
   - Segment: Customer segment (e.g., Consumer, Corporate)

5. Dim_Region.csv ({len(dim_region):,} rows, {len(dim_region.columns)} columns)
   Purpose: Geographic location information
   Key columns:
   - Region_Key: Primary key
   - City: City name
   - State: State/Province name
   - Region: Region name

IMPORT INTO POWER BI:
1. Click "Get Data" -> "Text/CSV"
2. Select all 5 CSV files
3. Load into Power BI
4. Create relationships as shown in diagram
5. Create measures from DAX_Measures.txt
"""

with open('model/CSV_Documentation.txt', 'w', encoding='utf-8') as f:
    f.write(csv_docs)

print("   ✅ CSV documentation saved: model/CSV_Documentation.txt")

# 10. FINAL OUTPUT

print("\n" + "="*60)
print("🎉 WEEK 2-3: DATA MODELING COMPLETE!")
print("="*60)

print("\n📁 DELIVERABLES CREATED:")
print("1. ✅ Star Schema Data Model")
print(f"   - Fact_Sales.csv ({len(fact_sales):,} rows, {len(fact_sales.columns)} columns)")
print(f"   - 4 Dimension tables created")

print("\n2. ✅ Data Model Documentation")
print("   - data_model_diagram.txt (ASCII diagram)")
print("   - simple_diagram.txt (Simplified diagram)")
print("   - DAX_Measures.txt (10+ measures)")
print("   - OnePage_Summary.txt (Project summary)")
print("   - README.txt (Complete documentation)")
print("   - CSV_Documentation.txt (File descriptions)")

print("\n3. ✅ DAX Measures Definitions")
print(f"   - Measures defined for: {available_measures}")
print("   - Includes SUM, AVG, YOY, MOM as required")

print("\n📊 DATA SUMMARY:")
print(f"   • Fact table columns: {list(fact_sales.columns)}")
print(f"   • Date range: {min_date} to {max_date}")
if 'Sales_Amount' in fact_sales.columns:
    print(f"   • Total Sales Value: ${fact_sales['Sales_Amount'].sum():,.2f}")

print("\n🚀 READY FOR WEEK 3 - DASHBOARD DEVELOPMENT:")
print("   Step 1: Load model/*.csv into Power BI")
print("   Step 2: Create relationships (star schema)")
print("   Step 3: Implement DAX measures")
print("   Step 4: Build Executive Summary dashboard")
print("   Step 5: Add slicers and interactivity")

print("\n✅ DATA MODELING PHASE SUCCESSFULLY COMPLETED!")
print("   All Week 2-3 requirements have been met.")

📊 STARTING DATA MODELING PHASE...

1. 📥 Loading cleaned data...
   ✅ Data loaded successfully: 9800 rows
   📋 Available columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Order_Year', 'Order_Month', 'Order_Quarter', 'Order_Day', 'Day_of_Week', 'Shipping_Days', 'Shipping_On_Time', 'Year_Month']

   🔍 Checking column availability...
   Available columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Order_Year', 'Order_Month', 'Order_Quarter', 'Order_Day', 'Day_of_Week', 'Shipping_Days', 'Shipping_On_Time', 'Year_Month']

   Converting date columns to datetime...
   Converted Order Date to datetime
   Converted Ship Date to date

In [10]:
import sqlite3
import pandas as pd
from IPython.display import display

# Connect to database
conn = sqlite3.connect('database/sales_database.db')

# Show all tables
print("📊 DATABASE TABLES:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
display(tables)

# Explore each table
for table in tables['name']:
    print(f"\n{'='*50}")
    print(f"📋 TABLE: {table}")
    print(f"{'='*50}")
    
    # Show structure
    structure = pd.read_sql_query(f"PRAGMA table_info({table})", conn)
    print(f"Columns ({len(structure)}):")
    for _, row in structure.iterrows():
        print(f"  - {row['name']} ({row['type']})")
    
    # Show row count
    count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn)
    print(f"Rows: {count['count'].iloc[0]:,}")
    
    # Show sample data
    if count['count'].iloc[0] > 0:
        sample = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 3", conn)
        print(f"Sample data:")
        display(sample)

# Close connection
conn.close()

📊 DATABASE TABLES:


,name


In [11]:
# auto_detect_warehouse.py
"""
AUTO-DETECTING DATA WAREHOUSE BUILDER
Team: Mariem & Rihem
This script automatically detects what columns exist in your data
"""

import sqlite3
import pandas as pd
import os

print("🤖 AUTO-DETECTING DATA STRUCTURE...")
print("=" * 60)

# 1. CHECK WHAT COLUMNS WE HAVE

print("\n📊 ANALYZING YOUR CSV FILES:")

# Check Fact_Sales columns
print("\n🔍 Fact_Sales.csv columns:")
fact_df = pd.read_csv('model/Fact_Sales.csv')
print("Columns:", list(fact_df.columns))
print("First row:", fact_df.iloc[0].to_dict())

# Check Dim_Product columns  
print("\n🔍 Dim_Product.csv columns:")
product_df = pd.read_csv('model/Dim_Product.csv')
print("Columns:", list(product_df.columns))

# 2. CREATE DATABASE WITH EXACT COLUMNS

print("\n🗄️  CREATING DATABASE WITH AUTO-DETECTED STRUCTURE...")

# Create database
db_path = 'database/sales_warehouse.db'
if os.path.exists(db_path):
    os.remove(db_path)  # Remove old database

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Function to clean column names
def clean_column_name(col):
    """Clean column names for SQL compatibility"""
    return str(col).lower().replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '').replace('.', '')

# Load Fact_Sales with auto-detected columns
print("\n📥 Loading Fact_Sales...")
fact_df = pd.read_csv('model/Fact_Sales.csv')

# Clean column names
fact_df.columns = [clean_column_name(col) for col in fact_df.columns]
print(f"   Cleaned columns: {list(fact_df.columns)}")
print(f"   Rows: {len(fact_df):,}")

# Save to database
fact_df.to_sql('fact_sales', conn, if_exists='replace', index=False)

# Load Dim_Date
print("\n📥 Loading Dim_Date...")
try:
    dim_date = pd.read_csv('model/Dim_Date.csv')
    dim_date.columns = [clean_column_name(col) for col in dim_date.columns]
    dim_date.to_sql('dim_date', conn, if_exists='replace', index=False)
    print(f"   ✅ Loaded: {len(dim_date):,} rows, {len(dim_date.columns)} columns")
except Exception as e:
    print(f"   ⚠️  Error loading Dim_Date: {e}")

# Load Dim_Product
print("\n📥 Loading Dim_Product...")
try:
    dim_product = pd.read_csv('model/Dim_Product.csv')
    dim_product.columns = [clean_column_name(col) for col in dim_product.columns]
    dim_product.to_sql('dim_product', conn, if_exists='replace', index=False)
    print(f"   ✅ Loaded: {len(dim_product):,} rows")
    print(f"   Columns: {list(dim_product.columns)}")
except Exception as e:
    print(f"   ⚠️  Error loading Dim_Product: {e}")

# Load Dim_Customer
print("\n📥 Loading Dim_Customer...")
try:
    dim_customer = pd.read_csv('model/Dim_Customer.csv')
    dim_customer.columns = [clean_column_name(col) for col in dim_customer.columns]
    dim_customer.to_sql('dim_customer', conn, if_exists='replace', index=False)
    print(f"   ✅ Loaded: {len(dim_customer):,} rows")
except Exception as e:
    print(f"   ⚠️  Error loading Dim_Customer: {e}")

# Load Dim_Region
print("\n📥 Loading Dim_Region...")
try:
    dim_region = pd.read_csv('model/Dim_Region.csv')
    dim_region.columns = [clean_column_name(col) for col in dim_region.columns]
    dim_region.to_sql('dim_region', conn, if_exists='replace', index=False)
    print(f"   ✅ Loaded: {len(dim_region):,} rows")
except Exception as e:
    print(f"   ⚠️  Error loading Dim_Region: {e}")


# 3. CHECK WHAT COLUMNS WE ACTUALLY HAVE IN DB

print("\n🔍 CHECKING DATABASE COLUMNS...")

# Get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()

table_columns = {}
for table_name, in tables:
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    column_names = [col[1] for col in columns]
    table_columns[table_name] = column_names
    
    print(f"\n📋 {table_name}:")
    for col_name in column_names:
        print(f"   • {col_name}")


# 4. CREATE DYNAMIC BUSINESS INSIGHTS

print("\n💡 CREATING BUSINESS INSIGHTS TABLE...")

# Build SELECT clause based on available columns
select_parts = []
group_by_parts = []

# Check what we have in fact_sales
fact_cols = table_columns.get('fact_sales', [])
print(f"\n📊 Available measures in fact_sales:")
for col in fact_cols:
    if col not in ['order_id', 'date_key', 'product_key', 'customer_key', 'region_key']:
        print(f"   • {col}")

# Time columns (from dim_date)
if 'dim_date' in table_columns:
    date_cols = table_columns['dim_date']
    if 'year' in date_cols:
        select_parts.append("d.year")
        group_by_parts.append("d.year")
    if 'quarter' in date_cols:
        select_parts.append("d.quarter")
        group_by_parts.append("d.quarter")
    if 'month_name' in date_cols:
        select_parts.append("d.month_name")
        group_by_parts.append("d.month_name")

# Product columns
if 'dim_product' in table_columns:
    prod_cols = table_columns['dim_product']
    if 'category' in prod_cols:
        select_parts.append("p.category")
        group_by_parts.append("p.category")
    if 'sub_category' in prod_cols or 'subcategory' in prod_cols:
        sub_col = 'sub_category' if 'sub_category' in prod_cols else 'subcategory'
        select_parts.append(f"p.{sub_col} as sub_category")
        group_by_parts.append(f"p.{sub_col}")

# Customer columns
if 'dim_customer' in table_columns:
    cust_cols = table_columns['dim_customer']
    if 'segment' in cust_cols:
        select_parts.append("c.segment")
        group_by_parts.append("c.segment")

# Region columns
if 'dim_region' in table_columns:
    region_cols = table_columns['dim_region']
    if 'region' in region_cols:
        select_parts.append("r.region")
        group_by_parts.append("r.region")
    if 'state' in region_cols:
        select_parts.append("r.state")
        group_by_parts.append("r.state")

# Build KPI SELECT clause
kpi_select = []
kpi_select.append("COUNT(f.order_id) as total_orders")

# Check for sales column (could be sales, sales_amount, amount, etc.)
sales_col = None
for col in ['sales_amount', 'sales', 'amount', 'revenue']:
    if col in fact_cols:
        sales_col = col
        break

if sales_col:
    kpi_select.append(f"SUM(f.{sales_col}) as total_sales")
    kpi_select.append(f"AVG(f.{sales_col}) as avg_order_value")

# Check for quantity column
if 'quantity' in fact_cols:
    kpi_select.append("SUM(f.quantity) as total_quantity")

# Check for profit column
profit_col = None
for col in ['profit_amount', 'profit', 'margin']:
    if col in fact_cols:
        profit_col = col
        break

if profit_col:
    kpi_select.append(f"SUM(f.{profit_col}) as total_profit")
    kpi_select.append(f"SUM(CASE WHEN f.{profit_col} > 0 THEN 1 ELSE 0 END) as profitable_orders")

# Check for discount column
if 'discount' in fact_cols:
    kpi_select.append("AVG(f.discount) as avg_discount")

# Check for shipping days
if 'shipping_days' in fact_cols:
    kpi_select.append("AVG(f.shipping_days) as avg_shipping_days")

# Build the complete query
if select_parts:  # Only create if we have dimension columns
    business_insights_query = f"""
    CREATE TABLE IF NOT EXISTS business_insights AS
    SELECT 
        {', '.join(select_parts)},
        {', '.join(kpi_select)}
    FROM fact_sales f
    LEFT JOIN dim_date d ON f.date_key = d.date_key
    LEFT JOIN dim_product p ON f.product_key = p.product_key
    LEFT JOIN dim_customer c ON f.customer_key = c.customer_key
    LEFT JOIN dim_region r ON f.region_key = r.region_key
    GROUP BY {', '.join(group_by_parts)}
    """
    
    print("\n📝 Generated query:")
    print(business_insights_query)
    
    try:
        cursor.execute("DROP TABLE IF EXISTS business_insights")
        cursor.execute(business_insights_query)
        print("✅ Created business_insights table!")
    except Exception as e:
        print(f"❌ Error creating business_insights: {e}")
else:
    print("⚠️  Not enough dimension columns to create business_insights table")


# 5. CREATE SIMPLE SUMMARY VIEWS

print("\n👁️  CREATING SIMPLE VIEWS FOR ANALYSIS...")

# Create monthly sales view
try:
    if sales_col and 'dim_date' in table_columns:
        cursor.execute(f"""
            CREATE VIEW IF NOT EXISTS vw_monthly_sales AS
            SELECT 
                d.year,
                d.month_name,
                COUNT(f.order_id) as total_orders,
                SUM(f.{sales_col}) as total_sales,
                AVG(f.{sales_col}) as avg_order_value
            FROM fact_sales f
            LEFT JOIN dim_date d ON f.date_key = d.date_key
            GROUP BY d.year, d.month_name
            ORDER BY d.year, d.month_name
        """)
        print("✅ Created vw_monthly_sales view")
except Exception as e:
    print(f"⚠️  Could not create monthly sales view: {e}")

# Create product performance view
try:
    if sales_col and 'dim_product' in table_columns:
        cursor.execute(f"""
            CREATE VIEW IF NOT EXISTS vw_product_performance AS
            SELECT 
                p.category,
                COUNT(f.order_id) as orders,
                SUM(f.{sales_col}) as sales,
                AVG(f.{sales_col}) as avg_sale_per_order
            FROM fact_sales f
            LEFT JOIN dim_product p ON f.product_key = p.product_key
            GROUP BY p.category
            ORDER BY sales DESC
        """)
        print("✅ Created vw_product_performance view")
except Exception as e:
    print(f"⚠️  Could not create product performance view: {e}")


# 6. FINAL SUMMARY

conn.commit()

print("\n📊 FINAL DATABASE SUMMARY:")
print("=" * 60)

cursor.execute("SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view') ORDER BY type, name;")
objects = cursor.fetchall()

for obj_type in ['table', 'view']:
    type_objs = [obj for obj in objects if obj[1] == obj_type]
    if type_objs:
        print(f"\n{obj_type.upper()}S:")
        for name, _ in type_objs:
            if obj_type == 'table':
                count = cursor.execute(f"SELECT COUNT(*) FROM {name}").fetchone()[0]
                print(f"   • {name} ({count:,} rows)")
            else:
                print(f"   • {name} (view)")

# Create a simple query to test
print("\n🧪 TEST QUERY - TOP 5 SALES BY DATE:")
try:
    if sales_col:
        test_query = f"""
        SELECT 
            d.year,
            d.month_name,
            COUNT(*) as orders,
            SUM(f.{sales_col}) as sales
        FROM fact_sales f
        LEFT JOIN dim_date d ON f.date_key = d.date_key
        GROUP BY d.year, d.month_name
        ORDER BY sales DESC
        LIMIT 5
        """
        test_result = pd.read_sql_query(test_query, conn)
        print(test_result.to_string(index=False))
except Exception as e:
    print(f"⚠️  Could not run test query: {e}")

conn.close()

print("\n" + "=" * 60)
print("🎉 DATA WAREHOUSE CREATED SUCCESSFULLY!")
print("=" * 60)
print(f"\n📁 Database file: {db_path}")
print(f"📊 Total tables created: {len([obj for obj in objects if obj[1] == 'table'])}")
print(f"👁️  Total views created: {len([obj for obj in objects if obj[1] == 'view'])}")
print("\n🚀 Your data is ready for Power BI and SQL queries!")

🤖 AUTO-DETECTING DATA STRUCTURE...

📊 ANALYZING YOUR CSV FILES:

🔍 Fact_Sales.csv columns:
Columns: ['Order_ID', 'Date_Key', 'Product_Key', 'Customer_Key', 'Region_Key', 'Sales_Amount', 'Shipping_Days']
First row: {'Order_ID': 'CA-2017-152156', 'Date_Key': 20171108, 'Product_Key': 1, 'Customer_Key': 1, 'Region_Key': 1, 'Sales_Amount': 261.96, 'Shipping_Days': 3}

🔍 Dim_Product.csv columns:
Columns: ['Product_Key', 'Product ID', 'Product Name', 'Category', 'Sub-Category']

🗄️  CREATING DATABASE WITH AUTO-DETECTED STRUCTURE...

📥 Loading Fact_Sales...
   Cleaned columns: ['order_id', 'date_key', 'product_key', 'customer_key', 'region_key', 'sales_amount', 'shipping_days']
   Rows: 9,800

📥 Loading Dim_Date...
   ✅ Loaded: 1,823 rows, 13 columns

📥 Loading Dim_Product...
   ✅ Loaded: 1,861 rows
   Columns: ['product_key', 'product_id', 'product_name', 'category', 'sub_category']

📥 Loading Dim_Customer...
   ✅ Loaded: 793 rows

📥 Loading Dim_Region...
   ✅ Loaded: 600 rows

🔍 CHECKING DAT